# Descriptive analytics

# Temporal Demand Patterns & Seasonality # Lina
- Pro Tag / Pro Woche / Pro Monat / Pro Jahr 
- Mit Wetterdaten vergleichen
 

## Geographical Demand Patterns # Nico, Jieyu
## ? Pro Stadtteil angemeldete Autos
## Altersgruppen je Stadtteil -> Zusammenhang zu Abo Modellen
## Pass -> Häufig für den Arbeitsweg? (RoundTrip, OneWay, Zeiten)

## KPIs
- Aktuelle Auslastung (Live & Historisch, Pro Station) (-> Mehr Stationen, Fahrräder?) # Lukas 
- Ausfall/Probleme: (Wartungsaufwand (z.B. in Prozent available/not available), ) # Ange
- Umsatz durch "Minutengeld" # Marc
- Anteil an Arten von Passhaltern?
- Anteil von Tripcategories?


In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import matplotlib
import matplotlib.pyplot as pp
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

import glob

%matplotlib inline

## preparing & cleaning data



In [ ]:
csv_files = glob.glob('Datasets/RideIndego 2016/*.csv')
df = pd.concat([pd.read_csv(f) for f  in csv_files], ignore_index = True)

### Find out NaN Values

In [ ]:
df.isna().sum()

### Drop any Row with NaN 0 in end_lat end_lon



In [ ]:
df['start_lat'] = pd.to_numeric(df['start_lat'], errors='coerce')
df['start_lon'] = pd.to_numeric(df['start_lon'], errors='coerce')
df['start_station_id'] = pd.to_numeric(df['start_station_id'], errors='coerce')
df['end_station_id'] = pd.to_numeric(df['end_station_id'], errors='coerce')
df['end_lat'] = pd.to_numeric(df['end_lat'], errors='coerce')
df['end_lon'] = pd.to_numeric(df['end_lon'], errors='coerce')
df['bike_id'] = pd.to_numeric(df['bike_id'], errors='coerce')
df['plan_duration'] = pd.to_numeric(df['plan_duration'], errors='coerce')

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna(how='any').reset_index(drop=True)
df

### Further Inspectation of data to prepare descriptive analysis (Marc)


In [ ]:
df.describe()

In [ ]:
df.info()

Alteration of start_time and end_time to datetime type

In [ ]:
df['start_time'] = pd.to_datetime(df['start_time'])

In [ ]:
df['end_time'] = pd.to_datetime(df['end_time'])

Calculate the length of a trip

In [ ]:
df["duration_trip"] = df["end_time"] - df["start_time"]

Now I am breaking the datetime of start_time into smaller parts(days, time, hour) to make it possible to visualize the usage over different periods. I am taking the start_time, not the end_time since it shows the demand.

In [ ]:
df['date'] = df['start_time'].dt.strftime('%m-%d')
#here something is not working correctly yet, has to be further examined

In [ ]:
df["time"] = df["start_time"].apply(lambda dt: dt.time)

In [ ]:
df["hour"] = df["start_time"].apply(lambda dt: dt.hour)

In [ ]:
df["weekday"] = df["start_time"].apply(lambda dt: dt.dayofweek)

In [ ]:
df["month"] = df["start_time"].apply(lambda dt: dt.month)

In [ ]:
df["full_date"] = [d.date() for d in df["start_time"]]


In [ ]:
df['date_hour'] = df['start_time'].dt.strftime('%m-%d-%H')


In [ ]:
#for a later purpose we will convert the duration from seconds to minutes, dividing it by 60:
df["duration"] = df["duration"] / 60

Inspection of new dataframe, uncomment if necessary

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
len(df.date_hour.unique())

### Including the weather data 

In [ ]:
#import weather data
#STR_Nov = pd.read_csv("Car2Go_STR_SampleData.csv", encoding = "ISO-8859-1")
weather = pd.read_csv('weather_hourly_philadelphia.csv')

In [ ]:
weather.head()

In [ ]:
weather.info()

In [ ]:
weather['date_time'] = pd.to_datetime(weather['date_time'])
weather['date'] = weather['date_time'].dt.strftime('%m-%d')
weather['hour'] = weather["date_time"].apply(lambda dt: dt.hour)

In [ ]:
weather['date_hour'] = weather['date_time'].dt.strftime('%m-%d-%H')
weather['year'] = pd.DatetimeIndex(weather['date_time']).year
weather = weather[weather['year'] == 2016]

#pd. DatetimeIndex(df['date']). year.
weather.head()

Combine both dataframes into one using mapping:

In [ ]:
# have to use something else than a full merge as it duplicates everything from df
df_weather = pd.merge(df, weather, on="date_hour", how="left") 

In [ ]:
# let us take another look at the merged dataframe:
df_weather.head()


# Task 2: Descriptive analytics

### a) Temporal Demand Patterns and Seasonality (Lina, Marc)

In [ ]:
#calculate the demand per hour
#hourly_demand = df.groupby(["hour"]).agg(demand=("bike_id", 'count'))
#hourly_demand = pd.DataFrame(hourly_demand)
#hourly_demand
hourly_demand = df.groupby(["date","hour"])["trip_id"].nunique()
hourly_demand = pd.DataFrame(hourly_demand)
hourly_demand


In [ ]:
fig, ax = pp.subplots(figsize=(10,4))

sns.barplot(x=hourly_demand.index.get_level_values(1), y=hourly_demand["trip_id"],ax=ax)
pp.xlabel('Hour', fontdict={'size':11})
pp.ylabel("Demand", fontdict={"size":11})
pp.title("Overview of hourly demand", fontsize=14.0, fontweight='bold')

pp.show()

In [ ]:
#demand per weekday:
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
#daily_demand = df.groupby(["weekday"]).agg(demand=("bike_id", 'count'))
daily_demand = df.groupby(["month","weekday"])["trip_id"].nunique()
daily_demand = pd.DataFrame(daily_demand)
daily_demand

In [ ]:
daily_demand.columns.values

In [ ]:
fig, ax = pp.subplots(figsize=(10,4))

sns.boxplot(x=daily_demand.index.get_level_values(1),y=daily_demand["trip_id"],palette = 'Blues_d')

pp.ylabel("Demand", fontsize=11)
pp.title("Use of bicycles in one week", fontsize=14, fontweight="bold", color="k")
pp.tight_layout()
pp.gca().set_xticklabels(\
    ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']);

pp.show()

In [ ]:
#calculate the monthly demand:
# different approach:
#monthly_demand = df.groupby(["month"]).agg(demand=("bike_id", 'count'))
monthly_demand = df.groupby(["month"])["trip_id"].nunique()

monthly_demand = pd.DataFrame(monthly_demand)
monthly_demand

In [ ]:
fig, ax = pp.subplots(figsize=(10,4))

sns.barplot(x=monthly_demand.index.get_level_values(0), y=monthly_demand["trip_id"],ax=ax)
pp.xlabel("Month", fontsize=11)
pp.ylabel("Demand", fontsize=11)
sns.set_color_codes("dark")
pp.title("The monthly use of bicycles ", fontsize=16, fontweight="bold", color="m")
pp.show()

### c) KPIs

Our first KPI is the current gross revenue which is calculated by the duration of the trip times $0,15 which is listed as the price/minute on Indego's website. We have to make some adjustments based depending on the type of pass the user has. For that we create a new dataframe only containing relevant data for us which includes the duration of the trip, the ID, the passtype, as well as some time info.

In [ ]:
price_per_minute = 0.15

In [ ]:
rev = df[['trip_id', 'duration', 'start_time', 'end_time', 'passholder_type', 'hour','date_hour']].copy()

In [ ]:
rev.head()

Since we take the price per minute and the duration of the trip is in seconds, we are going to divide the duration by 60 to get it in minutes.

In [ ]:
#rev["duration"] = rev["duration"]/60

In [ ]:
a = rev['passholder_type'].unique()
a

In [ ]:
rev['relevant_duration'] = rev.duration + rev.passholder_type.map( lambda x: -30 if x == 'Walk-up' else -60)

In [ ]:
rev.head()

In [ ]:
rev.loc[rev.relevant_duration <= 0, "relevant_duration"] = 0

In [ ]:
rev["rev_flex"] = rev["relevant_duration"] * price_per_minute

In [ ]:
rev.head()

In [ ]:
sum(rev["rev_flex"])

In [ ]:
hourly_rev = rev.groupby(["date_hour"]).agg(revenue=("rev_flex", 'sum'))
hourly_rev = pd.DataFrame(hourly_rev)
hourly_rev

In [ ]:
fig, ax = pp.subplots(figsize=(10,4))

sns.lineplot(data=hourly_rev, palette="tab10", linewidth=2.5)
pp.show()

Another KPI could be the share of walkups vs. passcard holder.

In [ ]:
type_shares = df[['passholder_type', 'date_hour']].copy()

In [ ]:
type_shares['type'] = type_shares.passholder_type.map( lambda x: 0 if x == 'Walk-up' else 1)

In [ ]:
type_shares.head()

In [ ]:
hourly_share = type_shares.groupby(["date_hour", "type"]).count()
hourly_share = pd.DataFrame(hourly_share)
hourly_share

In [ ]:
hourly_share_per = hourly_share.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

In [ ]:
hourly_share_per

#  Task 3: Predictive Analytics

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

## Philadelphia weather in 2016

In [ ]:
#calculate the weather over the year:
#different approach:
weather_month = df_weather.groupby(["month"]).agg(Temperature =("max_temp", 'mean'))
weather_month = pd.DataFrame(weather_month)
weather_month


In [ ]:
fig, ax = pp.subplots(figsize=(12,6))

sns.lineplot(x=weather_month.index.get_level_values(0), y=weather_month["Temperature"],ax=ax)
dim=np.arange(1,13,1)
pp.xticks(dim)
pp.xlabel("Months", fontsize=11)
pp.ylabel("Temperature", fontsize=11)
pp.title("2016: Weather in Philadelphia", fontsize=14, fontweight="bold", color="b")
pp.show()

### Demand and temperatures in comparison 

In [ ]:
fig, ax = pp.subplots(figsize=(12,6))

sns.barplot(x=monthly_demand.index.get_level_values(0), y=monthly_demand["trip_id"],ax=ax)
sns.set_color_codes("pastel")
ax2=ax.twinx()
sns.lineplot(x=weather_month.index.get_level_values(0), y=weather_month["Temperature"],ax=ax2, color="r", linewidth=2.5)
dim=np.arange(0,13,1)
pp.xticks(dim)

pp.show()

Future demand is a key factor that will steer operational decision making of a shared rental network. As a data scientist it is your responsibility to facilitate this type of decision support. For the purpose of this assignment we will be interested in forecasting total system-level demand in the next hour. To do so, develop a prediction model that predicts bike rental demand as a function of suitable features available in or derived from the datasets (incl. the weather data).

First of all, i am going to create a table that contains the demand of the hour along with some features as temperature, the hour of course, the day of the week and the time of the year.

In [ ]:
#new_df = df[["trip_id","weekday", "month"]].groupby(["weekday"])'

new_df_merged = pd.merge(weather, hourly_demand, on=["date", "hour"], how = "left")
new_df_merged["demand"] = new_df_merged["trip_id"]
new_df_merged["weekday"] = new_df_merged["date_time"].apply(lambda dt: dt.dayofweek)
new_df_merged["month"] = new_df_merged["date_time"].apply(lambda dt: dt.month)
new_df_merged

In [ ]:
#Drop columns that are not relevant for the regression
new_df_merged.drop(columns = ['trip_id', 'year'])

Here i check whether holidays like the 4th of July are having an impact on demand.

In [ ]:
july = new_df_merged[new_df_merged['month'] == 7]
july

In [ ]:
july_grouped = july.groupby(["date"])["demand"].sum()
july_grouped = pd.DataFrame(july_grouped)
july_grouped
#monthly_demand = df.groupby(["month"])["trip_id"].nunique()

#monthly_demand = pd.DataFrame(monthly_demand)
#monthly_demand

In [ ]:
fig, ax = pp.subplots(figsize=(20,4))

sns.lineplot(x=july_grouped.index.get_level_values(0), y=july_grouped["demand"],ax=ax)
pp.xlabel('Day', fontdict={'size':11})
pp.ylabel("Demand", fontdict={"size":11})
pp.title("Overview of hourly demand", fontsize=14.0, fontweight='bold')

pp.show()

In [ ]:
df_1 = new_df_merged
#df_1["summer"] = df_1[(df_1["month"] > 5) & (df_1["month"] < 12)]
#df_1["IsWeekday"] = df_1["weekday"].apply(lambda x: 1 if x<=4 else 0)
df_1["IsWeekday"] = df_1["weekday"].apply(lambda x: 1 if x<=4 else 0)

In [ ]:
df_1["summer"] = df_1["month"].apply(lambda x: 1 if (x> 5 & x<12) else 0)
df_1.head()


In [ ]:
X = df_1[["max_temp", "IsWeekday", "hour", "precip", "summer"]]
y = df_1[["demand"]]
y['demand'] = df_1['demand'].fillna(0)

In [ ]:
#Splitting the data to avoid overfitting and enable testing
features=list(zip(X["max_temp"],X["precip"], X["IsWeekday"], X["summer"], X["hour"]))
# Do a 70-30 split first
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.3,random_state=34 )

# now split X_train to achive 50-20-30 split
X_train, X_hold, y_train, y_hold = train_test_split(X_train, y_train, test_size=(0.2/0.7),random_state=34 )


In [ ]:
linear_model = LinearRegression(fit_intercept=True, normalize=False)

In [ ]:
linear_model.fit(X_train, y_train)
print(linear_model.coef_, linear_model.intercept_)

In [ ]:
model_L1.fit(X_train, y_train)
predict = model_L1.predict(X_test)
r2_score(y_test, predict)

Decession Tree Algorithm:

In [ ]:
#find the optimal tree depth
find_tree_depth (features,y)


In [ ]:
#Implement the algorithm
Tree_reg = DecisionTreeRegressor(max_depth=9)
tree_model = Tree_reg.fit(X_train, y_train) 

#Predict
y_hat_tree = tree_model.predict(X_test)

In [ ]:
print("R2:",r2_score(y_test, y_hat_tree))

In [ ]:
# YOUR CODE HERE:

def find_tree_depth (x,y):
    
    # define list for collecting results
    err_train = [] 
    err_test = []
    
    # split data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=10)
    
    #loop over max_depth
    
    for n in np.arange(1,21): # lets test until 24 for now
        
        # fit model
        
        tree_reg = DecisionTreeRegressor(max_depth=n)
        tree_model = tree_reg.fit(x_train,y_train)
        
        # compute errors
        
        err_train.append(mean_absolute_error(y_train, tree_model.predict(x_train)))
        err_test.append(mean_absolute_error(y_test, tree_model.predict(x_test)))


    pp.figure(figsize = (8,6))
    pp.plot(np.arange(1,21), err_train,np.arange(1,21), err_test)
    pp.legend(["Training", "Validation"])
    pp.xlabel("Max Tree Depth")
    pp.ylabel("MAE")
    pp.title("Search over max_depth parameter",fontsize=14)
    #plt.ylim((0,1))
    pp.show()




Now we are taking a look at the KNN Algorithm:

In [ ]:
KNN_reg = KNeighborsRegressor(n_neighbors=11)
KNN_model = KNN_reg.fit(X_train, y_train) 

# Predict
y_hat_KNN = KNN_model.predict(X_test)

In [ ]:
print("Test set performance:")

print("MAE:",mean_absolute_error(y_hat_KNN, y_test), "demand")
print("RMSE:",(mean_squared_error(y_hat_KNN, y_test))**(0.5), "demand")  
print("R2:",r2_score(y_test, y_hat_KNN))